In [42]:
import polars as pl
from pprint import pprint

In [43]:
def fix_str_to_int(data:pl.DataFrame, fix_list:list, type:any=pl.Int64):
    data = data.with_columns(pl.col(fix_list).str.split(","))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().str.strip_chars()))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().cast(pl.Int64, strict=False)))

    for column in fix_list:
        data = data.with_columns(pl.when(pl.col(column).list.eval(pl.element().is_null()).list.any()).then(None).otherwise(pl.col(column)).alias("bools"))
        data = data.select(replace(data, data.get_column_index(column), "bools")).rename({"bools":column})
    
    # data = data.with_columns(pl.col(fix_list).str.strip().str.replace("","0")).cast(conversion_dict)

    return data


# Abych nebyl nařčen z plagiátorství, zdroj: https://stackoverflow.com/questions/75954280/how-to-change-the-position-of-a-single-column-in-python-polars-library
def replace(df, new_position, col_name):
    neworder=df.columns
    neworder.pop(new_position)
    neworder.remove(col_name)
    neworder.insert(new_position,col_name)
    return neworder


In [44]:
rozvrh_by_kat = pl.read_csv("source_tables/rozvrhy_fakulta.csv").drop("semestr").unique().rename({"predmet" : "zkratka"})
rozvrh_by_kat = fix_str_to_int(rozvrh_by_kat, ["ucitIdno.ucitel", "vsichniUciteleUcitIdno"])
#print(rozvrh_by_kat.head())
predmety_by_kat = pl.read_csv("source_tables/predmety_fakulta.csv").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])
predmety_by_kat = fix_str_to_int(predmety_by_kat, ["garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "hodZaSemKombForma"])
predmety_by_kat.head(10)
#predmety_by_kat = pl.read_excel("source_tables/getPredmetyByKatedraFullInfo-2023-03-15-17-40.xlsx", sheet_id=1, engine="openpyxl").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""CNB""","""BP2""",2023,"""Bakalářská prá…","""Bakalářská prá…","""A""","""N""",5,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KFY/BP2""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet uděluj…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""CNB""","""KEXKU""",2023,"""Exkurze""","""Exkurze""","""A""","""A""",2,"""NE""","""""","""'RNDr. Regina …","""'RNDr. Regina …",[8317],"""""","""""",null,"""'RNDr. Regina …","""'RNDr. Regina …",[8317],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""DNU/SEM""",0,"""HOD/TYD""","""Náplní předmět…","""Zápočet""","""NE""","""Kombinovaná""","""Nutnou podmínk…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""CNB""","""NA06""",2023,"""Struk. látek a…","""Struktura láte…","""A""","""A""",3,"""NE""","""""","""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""' C Giaccova…","""KFY/NA06""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""NE""","""Kombinovaná""","""""","""Přednáška bude…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Mgr.""","""A""",[0],"""""","""""",""""""
"""CNB""","""KNM""",2023,"""Kompozitní a n…","""Kompozitní a n…","""N""","""A""",3,"""NE""","""""","""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Čapková P. Na…","""KFY/KNM""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Tento kurz sle…","""Zkouška""","""ANO""","""Kombinovaná""","""Seminární prác…","""1.-3. Úvod: vy…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""CNB""","""MSBA""",2023,"""Mikrofluidní s…","""Mikrofluidní s…","""N""","""A""",3,"""NE""","""""","""'Mgr. Marcel Š…","""'Mgr. Marcel Š…",[1503],"""'Mgr. Marcel Š…","""'Mgr. Marcel Š

In [45]:
# qr = ~pl.all().is_duplicated()
# rozvrh_by_kat = rozvrh_by_kat.filter(qr)
# predmety_by_kat = predmety_by_kat.filter(qr)


In [46]:
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""CNB""","""BP2""",2023,"""Bakalářská prá…","""Bakalářská prá…","""A""","""N""",5,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KFY/BP2""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet uděluj…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""CNB""","""KEXKU""",2023,"""Exkurze""","""Exkurze""","""A""","""A""",2,"""NE""","""""","""'RNDr. Regina …","""'RNDr. Regina …",[8317],"""""","""""",null,"""'RNDr. Regina …","""'RNDr. Regina …",[8317],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""DNU/SEM""",0,"""HOD/TYD""","""Náplní předmět…","""Zápočet""","""NE""","""Kombinovaná""","""Nutnou podmínk…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""CNB""","""NA06""",2023,"""Struk. látek a…","""Struktura láte…","""A""","""A""",3,"""NE""","""""","""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""' C Giaccova…","""KFY/NA06""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""NE""","""Kombinovaná""","""""","""Přednáška bude…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Mgr.""","""A""",[0],"""""","""""",""""""
"""CNB""","""KNM""",2023,"""Kompozitní a n…","""Kompozitní a n…","""N""","""A""",3,"""NE""","""""","""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Čapková P. Na…","""KFY/KNM""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Tento kurz sle…","""Zkouška""","""ANO""","""Kombinovaná""","""Seminární prác…","""1.-3. Úvod: vy…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""CNB""","""MSBA""",2023,"""Mikrofluidní s…","""Mikrofluidní s…","""N""","""A""",3,"""NE""","""""","""'Mgr. Marcel Š…","""'Mgr. Marcel Š…",[1503],"""'Mgr. Marcel Š…","""'Mgr. Marcel Š

In [47]:
rozvrh_by_kat.head(10)
#rozvrh_by_kat.__len__()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,str,str,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
469188,"""Úvod do supram…","""CNB""","""NA22""","""""","""3553""",[3553],"""Pavla""","""Čapková""","""prof. RNDr.""","""DrSc.""","""A""","""A""","""100""",2023,"""CP""","""2.07""","""2""",2,1,"""Cvičení""","""Cv""","""A""","""Úterý""","""Út""","""""","""7""","""7""",1,"""13:00""","""13:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N NANO""","""UJEP""","""""","""3.10.2023""","""2.1.2024""","""R""",[3553],"""prof. RNDr. Pa…","""'prof. RNDr. P…","""Čapková""",469188,"""""","""""","""PETRUSPRF""",""""""
476300,"""Výp. a odhad. …","""CNB""","""METD""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""2.32""","""22""",15,1,"""Přednáška""","""Př""","""A""","""Středa""","""St""","""""","""2""","""2""",1,"""08:00""","""08:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.NANO,2.NANO,…","""UJEP""","""""","""4.10.2023""","""3.1.2024""","""R""",[332],"""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská""",476300,"""""","""""","""PETRUSPRF""",""""""
469201,"""Principy, tech…","""CNB""","""PTVN""","""""","""4587""",[4587],"""Petr""","""Ryšánek""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""2.38""","""16""",5,5,"""Cvičení""","""Cv""","""A""","""Čtvrtek""","""Čt""","""""","""5""","""5""",1,"""11:00""","""11:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""3.NANO""","""UJEP""","""""","""5.10.2023""","""4.1.2024""","""R""",[4587],"""RNDr. Petr Ryš…","""'RNDr. Petr Ry…","""Ryšánek""",469201,"""""","""""","""PETRUSPRF""",""""""
476309,"""Metody příprav…","""CNB""","""M101""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.26""","""12""",5,4,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""4""","""4""",1,"""10:00""","""10:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N ACH""","""UJEP""","""""","""6.10.2023""","""5.1.2024""","""R""","[332, 280, 308]","""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská, Malý, …",476309,"""""","""""","""PETRUSPRF""",""""""
476308,"""Bakalářská prá…","""CNB""","""BP2""","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""",2023,"""""","""""","""""",10,4,"""Cvičení""","""Cv""","""A""","""""","""""","""""","""""","""""",0,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""3.NANO""","""""","""""","""""","""""","""R""",null,"""""","""""","""""",476308,"""""","""""","""PETRUSPRF""",""""""
484575,"""Fyzika a fyzik…","""CNB""","""FCHM""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""50""",2023,"""CP""","""2.33""","""8""",5,3,"""Přednáška""","""Př""","""A""","""Pondělí""","""Po""","""""","""2""","""3""",2,"""08:00""","""09:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.NANO""","""UJEP""","""""","""19.2.2024""","""13.5.2024""","""R""","[332, 9281]","""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská, Daniš""",484575,"""""","""""","""RSMEJKAL""","""

In [48]:
set(rozvrh_by_kat.columns) & set(predmety_by_kat.columns)

{'katedra', 'nazev', 'rok', 'zkratka'}

In [49]:
rozvrh_by_kat.null_count()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,164,238,0,0,0,0,0,0,164,0,0,66,160,0,0,0,0,0,0,0,0,474,474,171,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,238,0,0,0,0,0,0,0,0


In [50]:
predmety_by_kat.null_count()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,391,0,0,1700,0,0,1306,0,0,2798,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1931,0,0,0


In [51]:
# vybírám pouze předměty, které mají rozvrhovou akci

In [52]:
predmety_s_akci = predmety_by_kat.join(other=rozvrh_by_kat, on="zkratka", how="inner").select(predmety_by_kat.columns).unique().sort("zkratka")
str(len(predmety_s_akci)) + "   " + str(len(predmety_by_kat)) # ...what does this do
predmety_s_akci.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KGEO""","""0023""",2023,"""VK - Malá geog…","""Malá geografic…","""A""","""A""",1,"""ANO""","""""","""'RNDr. Ivan Fa…","""'RNDr. Ivan Fa…","[315, 3164]","""""","""""",null,"""'RNDr. Ivan Fa…","""'RNDr. Ivan Fa…","[315, 3164]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Lokálně zaměř…","""""","""""","""""","""A""",0,"""DNU/SEM""",1,"""DNU/SEM""",0,"""DNU/SEM""","""Jedno nebo dvo…","""Zápočet""","""NE""","""Ústní""","""Sledujte infor…","""1) stanovení f…","""Výuka v cizím …","""""","""""","""""","""Čeština, Němči…","""vk 1037,1537""","""N""","""N""","""N""","""""","""1""","""1""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0033""",2023,"""Optimalizace v…","""Optimalizace v…","""A""","""A""",2,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""""","""""",null,"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Lagová M., Ja…","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Náplní výběrov…","""Zápočet""","""NE""","""Ústní""","""Vstupní požada…","""1) Úvod do mat…","""""","""""","""""","""""","""Čeština""","""vk 1528""","""A""","""N""","""N""","""""","""1""","""1""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KGEO""","""0065""",2023,"""VK - Malá geog…","""Malá geografic…","""N""","""A""",1,"""ANO""","""""","""'RNDr. Ivan Fa…","""'RNDr. Ivan Fa…","[315, 3164]","""""","""""",null,"""'RNDr. Ivan Fa…","""'RNDr. Ivan Fa…","[315, 3164]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Lokálně zaměř…","""""","""""","""""","""A""",0,"""DNU/SEM""",1,"""DNU/SEM""",0,"""HOD/TYD""","""Exkurze je ved…","""Zápočet""","""NE""","""Ústní""","""aktivní účast,…","""1. Seznámení s…","""Výuka v cizím …","""""","""""","""""","""Čeština, Němči…","""vk""","""N""","""N""","""A""","""""","""1""","""1""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KGEO""","""0097""",2023,"""VK-Vybrané kap…","""Vybrané kapito…","""A""","""A""",2,"""NE""","""""","""'RNDr. Jiří Ma…","""'RNDr. Jiří Ma…",[828],"""""","""""",null,"""'RNDr. Jiří Ma…","""'RNDr. Jiří Ma…",[828],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""?Západní? Evro…","""Zápočet""","""NE""","""Kombinovaná""","""""","""1. Vymezení ?z…","""""","""""","""""","""""","""Čeština""","""""","""N""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KGEO""","""0099""",2023,"""VK - Geodetick…","

# Vyučující a garanti

In [53]:
jednotek_prednasek = pl.col("jednotekPrednasek")
jednotek_cviceni = pl.col("jednotekCviceni")
jednotek_seminare = pl.col("jednotekSeminare")
garant = pl.col("garanti")
cvicici = pl.col("cvicici")
prednasejici = pl.col("prednasejici")
seminarici = pl.col("seminarici")

### Každý předmět má právě jednoho garanta

> Zde jsou předměty, které nemají garanta:

In [54]:
zkratky = predmety_s_akci.filter(pl.col("garantiUcitIdno").is_null()).select(["zkratka", "rok", "nazev", "nazevDlouhy", "garanti", "garantiSPodily"]).filter(
    # Není SZ
    # pl.col("zkratka").str.starts_with("SZ").is_not()
    True
)
print(f"\n{len(zkratky)} Předmětů nemá zadaného garanta")
zkratky.write_excel("results/bez_garanta.xlsx")
zkratky
# print(list(zkratky), sep=", ")


163 Předmětů nemá zadaného garanta


zkratka,rok,nazev,nazevDlouhy,garanti,garantiSPodily
str,i64,str,str,str,str
"""0099""",2023,"""VK - Geodetick…","""Geodetické met…","""""",""""""
"""0121""",2023,"""VK-Propagace a…","""Propagace a pr…","""""",""""""
"""0165""",2023,"""Hodnotová orie…","""Hodnotová orie…","""""",""""""
"""0199""",2023,"""VK - Aplikace …","""Aplikace ATMEL…","""""",""""""
"""0201""",2023,"""VK-Praktikum m…","""VK-Praktikum m…","""""",""""""
…,…,…,…,…,…
"""SZZSŠ""",2023,"""Učitelství fyz…","""Učitelství fyz…","""""",""""""
"""SZZSŠ""",2023,"""Učitelství geo…","""Učitelství geo…","""""",""""""
"""SZZTD""",2023,"""Teoretické zák…","""Teoretické zák…","""""",""""""


> Předměty podezřelé z více garantů:

In [55]:
# predmety_s_akci.filter( > pl.lit())
# Nefunguje, arr.lengths údajně nic není
vice_garantu = predmety_s_akci.with_columns(pl.col("garantiUcitIdno").list.len().alias("pocet garantu")
    ).select(
        ["garantiUcitIdno", "garanti", "zkratka", "pocet garantu"]
    ).filter(pl.col("pocet garantu") > 1)
vice_garantu.write_excel("results/vice_garantu.xlsx")
vice_garantu.head(10)

garantiUcitIdno,garanti,zkratka,pocet garantu
list[i64],str,str,u32
"[315, 3164]","""'RNDr. Ivan Fa…","""0023""",2
"[315, 3164]","""'RNDr. Ivan Fa…","""0065""",2
"[1803, 305]","""'doc. Ing. Jar…","""B201""",2
"[3452, 3528]","""'doc. PhDr. RN…","""B600""",2
"[8441, 283]","""'doc. PhDr. Ka…","""BK107""",2
"[1999, 8280]","""'RNDr. Blanka …","""BK108""",2
"[4380, 5754]","""'Mgr. Jaroslav…","""K100""",2
"[304, 612]","""'doc. RNDr. Ma…","""K206""",2
"[3661, 283]","""'prof. doc. RN…","""K301""",2


### Každý předmět má alespoň jednoho vyučujícího v kategoriích dle parametrů předmětu

např. pokud má předmět rozsah 2 h přednášek a 2 h cvičení, pak musí mít alespoň jednoho přednášejícího a alespoň jednoho cvičícího a zároveň nesmí mít vyučujícího v kategorii vede seminář.

In [56]:
ma_prednasku = jednotek_prednasek != 0
nema_prednasejiciho = pl.col("prednasejiciUcitIdno").is_null()

chybi_prednasejici = predmety_s_akci.filter(ma_prednasku & nema_prednasejiciho)
chybi_prednasejici.write_excel("results/chybi_prednasejici.xlsx")
chybi_prednasejici.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KPRF""","""0101""",2023,"""Úvod do meteor…","""Úvod do meteor…","""A""","""N""",3,"""NE""","""""","""'doc. Mgr. Pav…","""'doc. Mgr. Pav…",[1434],"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""KPRF/0102""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Meteorologie j…","""Zápočet""","""NE""","""Kombinovaná""","""""","""1. Historie, s…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KGEO""","""0159""",2023,"""Smart City""","""Smart City""","""A""","""A""",2,"""NE""","""""","""'Mgr. Vladan H…","""'Mgr. Vladan H…",[4222],"""""","""""",null,"""'Mgr. Vladan H…","""'Mgr. Vladan H…","[4222, 7684]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Vanolo, A. (2…","""""","""""","""""","""A""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Smart city/reg…","""Zápočet""","""NE""","""Kombinovaná""","""prezence na cv…","""1. Konceptuali…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KGEO""","""B210""",2023,"""Euroregiony a …","""Euroregiony a …","""N""","""A""",4,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'JEŘÁBEK, M., …","""""","""""","""""","""N""",2,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zkouška""","""ANO""","""Kombinovaná""","""- odevzdání vš…","""1. Národ, náro…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""12""","""3""","""5"""
"""KGEO""","""B414""",2023,"""Ekonomická geo…","""Ekonomická geo…","""N""","""A""",4,"""NE""","""""","""'Mgr. Vladan H…","""'Mgr. Vladan H…",[4222],"""""","""""",null,"""'Mgr. Vladan H…","""'Mgr. Vladan H…",[4222],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Coe, N., Kell…","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz řeší prob…","""Zápočet""","""NE""","""Kombinovaná""","""Aktivní účast …","""1. Komoditní ř…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""20""","""10""","""1"""
"""KBI""","""BK201""",2023,"""Chemie a fyzik…","""Chemie a fyzik…","""A""","""N""",4,"""NE""","""""","""'RNDr. Oldřich…","""'RNDr. Oldřich…",[1983],"""""","""""",null,"""""","""""",nul

In [57]:
ma_cviceni = jednotek_cviceni != 0
nema_cviciciho = pl.col("cviciciUcitIdno").is_null()

chybi_cvicici = predmety_s_akci.filter(ma_cviceni & nema_cviciciho)
chybi_cvicici.write_excel("results/chybi_cvicici.xlsx")
chybi_cvicici.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KGEO""","""0099""",2023,"""VK - Geodetick…","""Geodetické met…","""A""","""N""",3,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Vinkler. F. G…","""""","""""","""""","""A""",0,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""Aktivní účast …","""Tematické zamě…","""""","""""","""""","""""","""Čeština""","""vk""","""N""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KGEO""","""0121""",2023,"""VK-Propagace a…","""Propagace a pr…","""A""","""A""",2,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'CLOW K., E., …","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je zaměře…","""Zápočet""","""NE""","""Kombinovaná""","""výborná práce …","""seznámením s m…","""""","""""","""""","""""","""Čeština""","""""","""N""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KGEO""","""0165""",2023,"""Hodnotová orie…","""Hodnotová orie…","""A""","""A""",2,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Patočka, J. D…","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""",""" Kurz je konci…","""Zápočet""","""NE""","""Kombinovaná""","""1. Aktivní úča…","""1. Přehled dis…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KFY""","""0199""",2023,"""VK - Aplikace …","""Aplikace ATMEL…","""A""","""A""",2,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KFY/0174""","""""","""""","""A""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KFY""","""0201""",2023,"""VK-Praktikum m…","""VK-Praktikum m…","""A""","""A""",2,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""

In [58]:
ma_seminare = jednotek_seminare != 0
nema_seminariciho = pl.col("seminariciUcitIdno").is_null()

chybi_seminarici = predmety_s_akci.filter(ma_seminare & nema_seminariciho)
chybi_seminarici.write_excel("results/chybi_seminarici.xlsx")
chybi_seminarici

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KCH""","""BK316""",2023,"""Laboratorní cv…","""Laboratorní cv…","""N""","""A""",4,"""NE""","""""","""'RNDr. Thu Huo…","""'RNDr. Thu Huo…",[429],"""""","""""",null,"""'Ing. Ivana Ka…","""'Ing. Ivana Ka…","[4226, 429]","""""","""""",null,"""""","""""","""""","""""","""KCH/BK301""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/SEM""",16,"""HOD/SEM""",1,"""HOD/SEM""","""Studenti se se…","""Zápočet""","""NE""","""Kombinovaná""","""""","""1. Důkaz příto…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""KAFJ""",2023,"""Teorie automat…","""Teorie automat…","""A""","""N""",5,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…","[449, 2220]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Chytil M. Aut…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""V tomto předmě…","""Zkouška""","""ANO""","""Kombinovaná""","""Požadavky pro …","""1. Konečné aut…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""","""0"""
"""KI""","""KANE""",2023,"""Analogová elek…","""Analogová elek…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""'doc. RNDr. Fr…","""'doc. RNDr. Fr…",[5232],"""""","""""",null,"""""","""""","""""","""""","""KI/KZEL""","""""","""""","""'Tichý, M. Ele…","""""","""""","""""","""N""",2,"""HOD/SEM""",4,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je zaměře…","""Zápočet""","""NE""","""Kombinovaná""","""Ústní obhajoba…","""1. Pasivní ele…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""KAPR1""",2023,"""Algoritmizace …","""Algoritmizace …","""A""","""N""",4,"""NE""","""""","""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","[6973, 650, … 6259]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/KAPR2, KI/K…","""'https://githu…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Úvodní kurz al…","""Zápočet""","""NE""","""Kombinovaná""","""Podmínkou získ…","""1. Základní te…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","

> Zde jsou předměty, ve kterých učitel vede seminář:

In [59]:
#prednasejici_je_seminarici = prednasejici == seminarici
#cvicici_je_seminarici = cvicici == seminarici

#konflikt_vyucujicich = predmety_s_akci.filter(ma_seminare & ma_prednasku & (prednasejici_je_seminarici | cvicici_je_seminarici))
#konflikt_vyucujicich.write_excel("results/konflikt_vyucujicich.xlsx")
#konflikt_vyucujicich.head()

In [60]:
print(f"{len(chybi_prednasejici)} předmětů nemá přednášejícího")
print(f"{len(chybi_cvicici)} předmětů nemá cvičícího")
print(f"{len(chybi_seminarici)} předmětů nemá seminářícího")
# print(f"U {len(konflikt_vyucujicich)} předmětů je seminářící také učitel")

31 předmětů nemá přednášejícího
84 předmětů nemá cvičícího
36 předmětů nemá seminářícího


### Garant by měl být někde mezi vyučujícími. Pokud má předmět přednášku, měl by být přednášející.

> V těchto předmětech garant vůbec neučí:

In [61]:
filter_query = (garant != cvicici) & (garant != seminarici) & (garant != prednasejici) & (ma_prednasku | ma_cviceni | ma_seminare)
predmety_kde_garant_neuci = predmety_s_akci.filter(filter_query)
predmety_kde_garant_neuci.write_excel("results/predmety_kde_garant_neuci.xlsx")
predmety_kde_garant_neuci.__len__()

543

> v těchto předmětech je přednáška, ale nedělá jí garant:

In [62]:
garant_neni_prednasejici = (jednotek_prednasek != 0) & (garant != prednasejici)
garant_neprednasi = predmety_s_akci.filter(garant_neni_prednasejici)
garant_neprednasi.write_excel("results/garant_neprednasi.xlsx")
garant_neprednasi.__len__()

307

## Kontrola proti rozvrhovaným akcím

### Vyučující u rozvrhových akcí musí být v příslušných kategoriích u vyučujících

#### Osekám dataframy

In [63]:
rozvrh_by_kat.head()
# predmety_by_kat.head()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,str,str,i64,i64,str,str,str,str,str,str,str,str,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
469188,"""Úvod do supram…","""CNB""","""NA22""","""""","""3553""",[3553],"""Pavla""","""Čapková""","""prof. RNDr.""","""DrSc.""","""A""","""A""","""100""",2023,"""CP""","""2.07""","""2""",2,1,"""Cvičení""","""Cv""","""A""","""Úterý""","""Út""","""""","""7""","""7""",1,"""13:00""","""13:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N NANO""","""UJEP""","""""","""3.10.2023""","""2.1.2024""","""R""",[3553],"""prof. RNDr. Pa…","""'prof. RNDr. P…","""Čapková""",469188,"""""","""""","""PETRUSPRF""",""""""
476300,"""Výp. a odhad. …","""CNB""","""METD""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""2.32""","""22""",15,1,"""Přednáška""","""Př""","""A""","""Středa""","""St""","""""","""2""","""2""",1,"""08:00""","""08:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.NANO,2.NANO,…","""UJEP""","""""","""4.10.2023""","""3.1.2024""","""R""",[332],"""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská""",476300,"""""","""""","""PETRUSPRF""",""""""
469201,"""Principy, tech…","""CNB""","""PTVN""","""""","""4587""",[4587],"""Petr""","""Ryšánek""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""2.38""","""16""",5,5,"""Cvičení""","""Cv""","""A""","""Čtvrtek""","""Čt""","""""","""5""","""5""",1,"""11:00""","""11:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""3.NANO""","""UJEP""","""""","""5.10.2023""","""4.1.2024""","""R""",[4587],"""RNDr. Petr Ryš…","""'RNDr. Petr Ry…","""Ryšánek""",469201,"""""","""""","""PETRUSPRF""",""""""
476309,"""Metody příprav…","""CNB""","""M101""","""""","""332""",[332],"""Zdeňka""","""Kolská""","""prof. Ing.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""","""3.26""","""12""",5,4,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""","""4""","""4""",1,"""10:00""","""10:50""",40,1,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.N ACH""","""UJEP""","""""","""6.10.2023""","""5.1.2024""","""R""","[332, 280, 308]","""prof. Ing. Zde…","""'prof. Ing. Zd…","""Kolská, Malý, …",476309,"""""","""""","""PETRUSPRF""",""""""
476308,"""Bakalářská prá…","""CNB""","""BP2""","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""",2023,"""""","""""","""""",10,4,"""Cvičení""","""Cv""","""A""","""""","""""","""""","""""","""""",0,"""00:00""","""00:00""",0,0,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""3.NANO""","""""","""""","""""","""""","""R""",null,"""""","""""","""""",476308,"""""","""""","""PETRUSPRF""",""""""


In [64]:
maly_rozvrh = rozvrh_by_kat.select(["zkratka", "vsichniUciteleUcitIdno", "typAkceZkr", "rok", "datum", "hodinaSkutOd", "hodinaSkutDo"])
maly_rozvrh = maly_rozvrh.with_columns(pl.col("vsichniUciteleUcitIdno")).explode("vsichniUciteleUcitIdno").rename({"vsichniUciteleUcitIdno": "idno"})
maly_rozvrh.head()

zkratka,idno,typAkceZkr,rok,datum,hodinaSkutOd,hodinaSkutDo
str,i64,str,i64,str,str,str
"""NA22""",3553,"""Cv""",2023,"""""","""13:00""","""13:50"""
"""METD""",332,"""Př""",2023,"""""","""08:00""","""08:50"""
"""PTVN""",4587,"""Cv""",2023,"""""","""11:00""","""11:50"""
"""M101""",332,"""Př""",2023,"""""","""10:00""","""10:50"""
"""M101""",280,"""Př""",2023,"""""","""10:00""","""10:50"""


In [65]:
# predmety_by_kat

In [66]:
from pprint import pprint
# pprint(predmety_by_kat.columns,compact=True)
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""CNB""","""BP2""",2023,"""Bakalářská prá…","""Bakalářská prá…","""A""","""N""",5,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""KFY/BP2""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem kurzu je…","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet uděluj…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""CNB""","""KEXKU""",2023,"""Exkurze""","""Exkurze""","""A""","""A""",2,"""NE""","""""","""'RNDr. Regina …","""'RNDr. Regina …",[8317],"""""","""""",null,"""'RNDr. Regina …","""'RNDr. Regina …",[8317],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",2,"""DNU/SEM""",0,"""HOD/TYD""","""Náplní předmět…","""Zápočet""","""NE""","""Kombinovaná""","""Nutnou podmínk…","""""","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""CNB""","""NA06""",2023,"""Struk. látek a…","""Struktura láte…","""A""","""A""",3,"""NE""","""""","""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""' C Giaccova…","""KFY/NA06""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Předmět poskyt…","""Zkouška""","""NE""","""Kombinovaná""","""""","""Přednáška bude…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Mgr.""","""A""",[0],"""""","""""",""""""
"""CNB""","""KNM""",2023,"""Kompozitní a n…","""Kompozitní a n…","""N""","""A""",3,"""NE""","""""","""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""'prof. RNDr. P…","""'prof. RNDr. P…",[3553],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Čapková P. Na…","""KFY/KNM""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Tento kurz sle…","""Zkouška""","""ANO""","""Kombinovaná""","""Seminární prác…","""1.-3. Úvod: vy…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""CNB""","""MSBA""",2023,"""Mikrofluidní s…","""Mikrofluidní s…","""N""","""A""",3,"""NE""","""""","""'Mgr. Marcel Š…","""'Mgr. Marcel Š…",[1503],"""'Mgr. Marcel Š…","""'Mgr. Marcel Š

In [67]:
ciselnik_ucitelu = pl.read_csv("source_tables/ciselnik_ucitelu.csv").select("nazev", "key").rename({"nazev":"jmena", "key":"idno"})
ciselnik_ucitelu.head()

jmena,idno
str,i64
"""Abrahamová Pet…",8409
"""Ač Alexander, …",9316
"""Adamec Slavomí…",5432
"""Adámek Pavel, …",7443
"""Adam Julie, M.…",9249


In [68]:
male_predmety = predmety_by_kat.select(pl.col(["zkratka", "katedra", "rok", "nazev", "garantiUcitIdno", "prednasejici", "prednasejiciUcitIdno","cvicici", "cviciciUcitIdno","seminarici", "seminariciUcitIdno"]).filter(pl.col("zkratka").is_unique()))
#male_predmety = male_predmety.with_columns(pl.col(["prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "garantiUcitIdno"]).str.split(", "))
male_predmety.filter(pl.col("zkratka") == "EPCA").head()

zkratka,katedra,rok,nazev,garantiUcitIdno,prednasejici,prednasejiciUcitIdno,cvicici,cviciciUcitIdno,seminarici,seminariciUcitIdno
str,str,i64,str,list[i64],str,list[i64],str,list[i64],str,list[i64]


In [69]:
# přednášející nemá přednášku:
filtrovani_prednasejici = male_predmety.select("nazev", "zkratka", "prednasejiciUcitIdno").explode("prednasejiciUcitIdno")
prednasejici_jmena = male_predmety.select("prednasejici").rename({"prednasejici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
prednasejici_jmena = prednasejici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_prednasejici = filtrovani_prednasejici.with_columns(prednasejici_jmena).filter(
    pl.col("prednasejiciUcitIdno"
).is_not_null()).with_columns(
    pl.col("prednasejiciUcitIdno").alias("idno")
)
joined_prednasejici = filtrovani_prednasejici.join(maly_rozvrh, "idno", "left")
prednasejici_bez_prednasek = joined_prednasejici.filter(pl.col("typAkceZkr").is_null())
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_excel("results/prednasejici_bez_prednasek.xlsx")
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno", "jmena").sort("prednasejiciUcitIdno").write_csv("results_testing/prednasejici_bez_prednasek.csv")

In [70]:
male_prednasky = maly_rozvrh.filter(pl.col("typAkceZkr") == "Př")
joined_prednasky = male_prednasky.join(male_predmety.select("zkratka", "katedra", "nazev", "prednasejici", "prednasejiciUcitIdno"), "zkratka", "left").unique()
prednasky_bez_prednasejicich = joined_prednasky.filter(pl.col("idno").is_in(pl.col("prednasejiciUcitIdno")).not_())

prednasky_bez_prednasejicich = prednasky_bez_prednasejicich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("prednasejiciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

prednasky_bez_prednasejicich.write_csv("results_testing/prednasky_bez_prednasejicich.csv")
prednasky_bez_prednasejicich.write_excel("results/prednasky_bez_prednasejicich.xlsx")


print(prednasky_bez_prednasejicich.__len__())
prednasky_bez_prednasejicich.head(15)

436


zkratka,idno,typAkceZkr,rok,datum,hodinaSkutOd,hodinaSkutDo,katedra,nazev,prednasejici,prednasejiciUcitIdno,jmena
str,i64,str,i64,str,str,str,str,str,str,str,str
"""0101""",556,"""Př""",2023,"""""","""16:00""","""17:50""",null,null,null,null,null
"""0159""",7684,"""Př""",2023,"""""","""00:00""","""00:00""",null,null,null,null,"""Smutná Zdeňka,…"
"""0160""",4310,"""Př""",2023,"""""","""12:00""","""12:50""",null,null,null,null,"""Šícha Václav, …"
"""0166""",7684,"""Př""",2023,"""""","""13:00""","""13:50""",null,null,null,null,"""Smutná Zdeňka,…"
"""0201""",8419,"""Př""",2023,"""""","""08:00""","""09:50""",null,null,null,null,"""Baszczyňski On…"
…,…,…,…,…,…,…,…,…,…,…,…
"""AP05""",null,"""Př""",2023,"""""","""00:00""","""00:00""","""KFY""","""Metody analýz …","""'RNDr. Jan Lor…","""1590""",null
"""AP15""",5431,"""Př""",2023,"""""","""07:00""","""07:50""",null,null,null,null,"""Henych Jiří, I…"
"""B100""",2855,"""Př""",2023,"""""","""12:00""","""13:50""",null,null,null,null,"""Jirsák Jan, do…"


In [71]:
# cvičící nemá cvičení:
filtrovani_cvicici = male_predmety.select("nazev", "zkratka", "cviciciUcitIdno").explode("cviciciUcitIdno")
cvicici_jmena = male_predmety.select("cvicici").rename({"cvicici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
cvicici_jmena = cvicici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_cvicici = filtrovani_cvicici.with_columns(cvicici_jmena).filter(
    pl.col("cviciciUcitIdno"
).is_not_null()).with_columns(
    pl.col("cviciciUcitIdno").alias("idno")
)
joined_cvicici = filtrovani_cvicici.join(maly_rozvrh, "idno", "left")
cvicici_bez_cviceni = joined_cvicici.filter(pl.col("typAkceZkr").is_null())
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno").sort("cviciciUcitIdno").write_excel("results/cvicici_bez_cviceni.xlsx")
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno", "jmena").sort("cviciciUcitIdno").write_csv("results_testing/cvicici_bez_cviceni.csv")

In [72]:
male_cviceni = maly_rozvrh.filter(pl.col("typAkceZkr") == "Cv")
joined_cviceni = male_cviceni.join(male_predmety.select("zkratka", "katedra", "nazev", "cvicici", "cviciciUcitIdno"), "zkratka", "left").unique()
cviceni_bez_cvicich = joined_cviceni.filter(pl.col("idno").is_in(pl.col("cviciciUcitIdno")).not_())

cviceni_bez_cvicich = cviceni_bez_cvicich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("cviciciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

cviceni_bez_cvicich.write_csv("results_testing/cviceni_bez_cvicich.csv")
cviceni_bez_cvicich.write_excel("results/cviceni_bez_cvicich.xlsx")

print(cviceni_bez_cvicich.__len__())
cviceni_bez_cvicich.head(15)

711


zkratka,idno,typAkceZkr,rok,datum,hodinaSkutOd,hodinaSkutDo,katedra,nazev,cvicici,cviciciUcitIdno,jmena
str,i64,str,i64,str,str,str,str,str,str,str,str
"""0023""",315,"""Cv""",2023,"""""","""00:00""","""00:00""",null,null,null,null,"""Farský Ivan, R…"
"""0033""",449,"""Cv""",2023,"""""","""14:00""","""15:50""",null,null,null,null,"""Barilla Jiří, …"
"""0097""",828,"""Cv""",2023,"""""","""12:00""","""13:50""",null,null,null,null,null
"""0099""",4303,"""Cv""",2023,"""""","""00:00""","""00:00""","""KGEO""","""VK - Geodetick…","""""",null,"""Dolejš Martin,…"
"""0104""",2856,"""Cv""",2023,"""""","""08:00""","""09:50""",null,null,null,null,"""Škvorová Magda…"
…,…,…,…,…,…,…,…,…,…,…,…
"""0142""",3452,"""Cv""",2023,"""""","""16:00""","""17:50""",null,null,null,null,"""Bláha Jan Dani…"
"""0159""",7684,"""Cv""",2023,"""""","""00:00""","""00:00""",null,null,null,null,"""Smutná Zdeňka,…"
"""0160""",8431,"""Cv""",2023,"""""","""12:00""","""13:50""",null,null,null,null,"""Štěbetáková Mi…"


In [73]:
# seminařicí nemá seminář:
filtrovani_seminarici = male_predmety.select("nazev", "zkratka", "seminariciUcitIdno").explode("seminariciUcitIdno")
seminarici_jmena = male_predmety.select("seminarici").rename({"seminarici":"jmena"}).with_columns(pl.col("jmena").str.strip_chars().str.split("', ")).explode("jmena")
seminarici_jmena = seminarici_jmena.with_columns(pl.col("jmena").str.replace(",", ""))
filtrovani_seminarici = filtrovani_seminarici.with_columns(seminarici_jmena).filter(
    pl.col("seminariciUcitIdno"
).is_not_null()).with_columns(
    pl.col("seminariciUcitIdno").alias("idno")
)
joined_seminarici = filtrovani_seminarici.join(maly_rozvrh, "idno", "left")
seminarici_bez_seminare = joined_seminarici.filter(pl.col("typAkceZkr").is_null())
seminarici_bez_seminare.select("nazev", "zkratka", "seminariciUcitIdno").sort("seminariciUcitIdno").write_excel("results/seminarici_bez_seminare.xlsx")

In [74]:
male_seminare = maly_rozvrh.filter(pl.col("typAkceZkr") == "Se")
joined_seminare = male_seminare.join(male_predmety.select("zkratka", "katedra", "nazev", "seminarici", "seminariciUcitIdno"), "zkratka", "left").unique()
seminare_bez_seminaricich = joined_seminare.filter(pl.col("idno").is_in(pl.col("seminariciUcitIdno")).not_()).unique()

seminare_bez_seminaricich = seminare_bez_seminaricich.join(ciselnik_ucitelu, "idno", "left").with_columns(pl.col("seminariciUcitIdno").cast(pl.List(pl.Utf8)).list.join(", ")).sort("zkratka")

seminare_bez_seminaricich.write_csv("results_testing/seminare_bez_seminaricich.csv")
seminare_bez_seminaricich.write_excel("results/seminare_bez_seminaricich.xlsx")
print(seminare_bez_seminaricich.__len__())
seminare_bez_seminaricich.head(6)

242


zkratka,idno,typAkceZkr,rok,datum,hodinaSkutOd,hodinaSkutDo,katedra,nazev,seminarici,seminariciUcitIdno,jmena
str,i64,str,i64,str,str,str,str,str,str,str,str
"""0131""",449,"""Se""",2023,"""""","""14:00""","""15:50""",null,null,null,null,"""Barilla Jiří, …"
"""0162""",7684,"""Se""",2023,"""""","""00:00""","""00:00""",null,null,null,null,"""Smutná Zdeňka,…"
"""0162""",8431,"""Se""",2023,"""""","""00:00""","""00:00""",null,null,null,null,"""Štěbetáková Mi…"
"""0201""",2317,"""Se""",2023,"""""","""12:00""","""13:50""",null,null,null,null,"""Kuba Pavel, In…"
"""0204""",null,"""Se""",2023,"""""","""15:00""","""16:50""",null,null,null,null,null
"""0204""",null,"""Se""",2023,"""""","""15:00""","""15:50""",null,null,null,null,null


In [75]:
predmety_by_kat.filter(pl.col("zkratka") == "KTZI2").head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""KTZI2""",2023,"""Teoretické zák…","""Teoretické zák…","""N""","""A""",3,"""NE""","""""","""'RNDr. Martin …","""'RNDr. Martin …",[264],"""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""'RNDr. Martin …","""'RNDr. Martin …","[264, 267]","""""","""""",null,"""""","""""","""""","""""","""KI/KTZI1""","""""","""""","""'Demel, J. Gra…","""""","""""","""""","""N""",6,"""HOD/SEM""",4,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je koncip…","""Zápočet""","""NE""","""Kombinovaná""","""Požadavky k zá…","""1. Rekurence a…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""


### warning, když je mezi vyučujícími někdo bez odpovídajícího typu rozvrhové akce

Pro každého učitele vypíšu předměty, u kterých má mít nějakou akci.

Takže budu mít dataframe, kde bude učitel, zkratka, a jeden sloupeček za každý typ akce, kde učitel bude muset mít aspoň 1 narozvrhovaný typ akce s daným předmětem, kde je hodnota `True`


In [76]:
# vypíšu typy akcí
list(maly_rozvrh.select(pl.col("typAkceZkr").unique()))

[shape: (3,)
 Series: 'typAkceZkr' [str]
 [
 	"Cv"
 	"Se"
 	"Př"
 ]]

### garant předmětu by měl být někde mezi vyučujícími rozvrhových akcí a pokud má předmět přednášku, měl by být i mezi přednášejícími v rámci rozvrhových akcí

### Změny proti akreditacím.

### Porovnat revize předmětů

tj. parametry předmětů pro aktuální akademický rok a pro příští.

> jakou to má mít podobu?